In [4]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler



In [5]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
df = pd.read_csv(
    Path('Resources/logistic_regression.csv')   
)

# Review the DataFrame
df.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,...,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,address
0,10000.0,36 months,11.44,329.48,B,B4,Marketing,10+ years,RENT,117000.0,...,16.0,0.0,36369.0,41.8,25.0,w,INDIVIDUAL,0.0,0.0,"0174 Michelle Gateway\nMendozaberg, OK 22690"
1,8000.0,36 months,11.99,265.68,B,B5,Credit analyst,4 years,MORTGAGE,65000.0,...,17.0,0.0,20131.0,53.3,27.0,f,INDIVIDUAL,3.0,0.0,"1076 Carney Fort Apt. 347\nLoganmouth, SD 05113"
2,15600.0,36 months,10.49,506.97,B,B3,Statistician,< 1 year,RENT,43057.0,...,13.0,0.0,11987.0,92.2,26.0,f,INDIVIDUAL,0.0,0.0,"87025 Mark Dale Apt. 269\nNew Sabrina, WV 05113"
3,7200.0,36 months,6.49,220.65,A,A2,Client Advocate,6 years,RENT,54000.0,...,6.0,0.0,5472.0,21.5,13.0,f,INDIVIDUAL,0.0,0.0,"823 Reid Ford\nDelacruzside, MA 00813"
4,24375.0,60 months,17.27,609.33,C,C5,Destiny Management Inc.,9 years,MORTGAGE,55000.0,...,13.0,0.0,24584.0,69.8,43.0,f,INDIVIDUAL,1.0,0.0,"679 Luna Roads\nGreggshire, VA 11650"


Data dictionary:

loan_amnt : The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.
term : The number of payments on the loan. Values are in months and can be either 36 or 60.
int_rate : Interest Rate on the loan
installment : The monthly payment owed by the borrower if the loan originates.
grade : LoanTap assigned loan grade
sub_grade : LoanTap assigned loan subgrade
emp_title :The job title supplied by the Borrower when applying for the loan.*
emp_length : Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years.
home_ownership : The home ownership status provided by the borrower during registration or obtained from the credit report.
annual_inc : The self-reported annual income provided by the borrower during registration.
verification_status : Indicates if income was verified by LoanTap, not verified, or if the income source was verified
issue_d : The month which the loan was funded
loan_status : Current status of the loan - Target Variable
purpose : A category provided by the borrower for the loan request.
title : The loan title provided by the borrower
dti : A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LoanTap loan, divided by the borrower’s self-reported monthly income.
earliest_cr_line :The month the borrower's earliest reported credit line was opened
open_acc : The number of open credit lines in the borrower's credit file.
pub_rec : Number of derogatory public records
revol_bal : Total credit revolving balance
revol_util : Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.
total_acc : The total number of credit lines currently in the borrower's credit file
initial_list_status : The initial listing status of the loan. Possible values are – W, F
application_type : Indicates whether the loan is an individual application or a joint application with two co-borrowers
mort_acc : Number of mortgage accounts.
pub_rec_bankruptcies : Number of public record bankruptcies
Address: Address of the individual

In [6]:
#drop columns that are not relevant to our analysis
clean_df = df.drop(columns =['verification_status','emp_title','title','initial_list_status','address','issue_d','pub_rec','earliest_cr_line'])
clean_df.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,loan_status,purpose,dti,open_acc,revol_bal,revol_util,total_acc,application_type,mort_acc,pub_rec_bankruptcies
0,10000.0,36 months,11.44,329.48,B,B4,10+ years,RENT,117000.0,Fully Paid,vacation,26.24,16.0,36369.0,41.8,25.0,INDIVIDUAL,0.0,0.0
1,8000.0,36 months,11.99,265.68,B,B5,4 years,MORTGAGE,65000.0,Fully Paid,debt_consolidation,22.05,17.0,20131.0,53.3,27.0,INDIVIDUAL,3.0,0.0
2,15600.0,36 months,10.49,506.97,B,B3,< 1 year,RENT,43057.0,Fully Paid,credit_card,12.79,13.0,11987.0,92.2,26.0,INDIVIDUAL,0.0,0.0
3,7200.0,36 months,6.49,220.65,A,A2,6 years,RENT,54000.0,Fully Paid,credit_card,2.60,6.0,5472.0,21.5,13.0,INDIVIDUAL,0.0,0.0
4,24375.0,60 months,17.27,609.33,C,C5,9 years,MORTGAGE,55000.0,Charged Off,credit_card,33.95,13.0,24584.0,69.8,43.0,INDIVIDUAL,1.0,0.0


In [7]:
#Deleting rows with null values
columns_to_check = ['revol_util', 'mort_acc','pub_rec_bankruptcies','emp_length']
clean_df.dropna(subset=columns_to_check, inplace=True)


In [8]:
#Store the dataframe in sqlite
from sqlalchemy import create_engine
engine = create_engine('sqlite:///SQL/Loan_data.db')
clean_df.to_sql('Loan_data', con=engine, index=False)

340775

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
categorical_columns =["term","grade","sub_grade","emp_length","home_ownership","purpose","application_type"]

df_dummies = pd.get_dummies(clean_df[categorical_columns])

# Merge the encoded columns back to the original DataFrame
clean_df = pd.concat([clean_df, df_dummies], axis=1)

# Drop the original categorical
clean_df.drop(categorical_columns, axis=1, inplace=True)


In [10]:
#if the loan was paid the value is changed to 1 if the loan was charged off the value is changed to 0
clean_df['loan_status'] = clean_df['loan_status'].replace({'Fully Paid': 1, 'Charged Off': 0})

C:\Users\diego\AppData\Local\Temp\ipykernel_4348\3429418315.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  clean_df['loan_status'] = clean_df['loan_status'].replace({'Fully Paid': 1, 'Charged Off': 0})


In [23]:
# Split our preprocessed data into our features and target arrays

y = clean_df["loan_status"].values
X = clean_df.drop(columns=["loan_status"]).values

# Split the preprocessed data into a training and testing dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [24]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [25]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
logistic_regression_model = LogisticRegression(random_state=78)

# Fit the model using training data
lr_model = logistic_regression_model.fit(X_train_scaled, y_train)

In [26]:
# Make a prediction using the testing data
testing_predictions = logistic_regression_model.predict(X_test_scaled)

In [27]:
# Generate a confusion matrix for the model
testing_matrix = confusion_matrix(y_test, testing_predictions)

In [28]:
# Print the classification report for the model
#TP: 1138 (True Positives)
#FN: 15800 (False Positives)
#FP: 936 (False Negatives)
#TN: 67320 (True Negatives)
print(testing_matrix)

[[ 1138 15800]
 [  936 67320]]


In [29]:
#Calculating accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, testing_predictions)
print("Accuracy:", accuracy)


Accuracy: 0.8035542409089842
